In [13]:

import numpy as np
import pandas as pd
fixedItemSet=np.load(r"hetrec2011-lastfm-2k\frequentItemVec.npy")
userflow=np.load(r'hetrec2011-lastfm-2k\userflow.npy')
userclick=np.load(r'hetrec2011-lastfm-2k\userclick.npy',allow_pickle=True)

def hwithoutconstraints(action):
    if type(action)==torch.Tensor:
        action=action.detach().numpy()
    items=np.argwhere(np.array(action)>0.5).reshape(args.card)
    if iteration<args.card:
        itemlist=userclick[0][:args.card]
    else:
        itemlist=userclick[0][(iteration-args.card):min((iteration+args.card),len(userclick[0]))]
    cnt2=0
    for i in items:
        cnt2+=( itemlist.count(i))
    return cnt2/(args.card)

import argparse
from copy import deepcopy
import torch
import gym
import time
import torch
import torch.nn as nn	
import torch.nn.functional as F	
import torch.optim as optim	
import numpy as np

import argparse
import sys

import copy
import numpy as np
from numpy.random import permutation
import matplotlib.pyplot as plt
import sys
sys.path.insert(0,'/root/gurobi903/linux64/lib/python3.6_utf32/gurobipy')

class para:
    def __init__(self):
        self.batch_size=16
        self.wmc=10
        self.solDim=40
        self.card=10
        self.test_batch_size=1000
        self.epochs=10
        self.lr=0.1
        self.momentum=0.5
        self.no_cuda='store_true'
        self.seed=1
        self.log_interval=10
        self.hidden_dim=10
args=para()
thre=1
legalList={}
for i in range(40):
    legalList[i]=[]
for i in range(40):
    for j in range(40):
        if i!=j and np.linalg.norm(fixedItemSet[i]-fixedItemSet[j])<thre:
            legalList[i].append(j)

In [14]:

import argparse
from copy import deepcopy
import torch
import gym
import time
import torch
import torch.nn as nn	
import torch.nn.functional as F	
import torch.optim as optim	
import numpy as np

import argparse
import sys

import copy
import numpy as np
from numpy.random import permutation
import matplotlib.pyplot as plt
import sys
sys.path.insert(0,'/root/gurobi903/linux64/lib/python3.6_utf32/gurobipy')


def checkFea(x):
    cnt=0
    cntT=0
    for i in range(len(legalList.keys())):
        for j in legalList[i]:
            cntT+=1
            if x[j]+x[i]>1:
                cnt+=1
    return cnt/cntT/2+abs(sum(x)-args.card)/args.card/2
def CB(alpha,x,M):
    return alpha*np.sqrt(np.dot(np.dot(x.T,np.linalg.inv(M)),x))
def solver(theta,MODEL):

    import gurobipy
    import time
    s=time.time()
    
    variables=[]
    for i in range(args.solDim):
        variables.append(MODEL.addVar(vtype=gurobipy.GRB.BINARY, name='x'+'i'))
    
    MODEL.update()

    
    MODEL.setObjective(np.array(variables).dot(theta), sense=gurobipy.GRB.MAXIMIZE)

    
    cnt=0
    for i in range(args.solDim):
        if len(legalList)>0:
            for j in legalList[i]:
                MODEL.addConstr(-variables[i]-variables[j] >= -1, name=str(cnt))
                cnt+=1
    MODEL.addConstr(sum(variables) == args.card, name=str(cnt))
    f=time.time()
    
    MODEL.optimize()
    return np.array(MODEL.x)
def solver_quad(Q,MODEL):

    import gurobipy
    import time
    s=time.time()
    
    variables=[]
    for i in range(args.solDim):
        variables.append(MODEL.addVar(vtype=gurobipy.GRB.BINARY, name='x'+'i'))
    
    MODEL.update()

    
    MODEL.setObjective(np.array(variables).dot(Q).dot(np.array(variables)), sense=gurobipy.GRB.MAXIMIZE)

    
    cnt=0
    for i in range(args.solDim):
        if len(legalList)>0:
            for j in legalList[i]:
                MODEL.addConstr(-variables[i]-variables[j] >= -1, name=str(cnt))
                cnt+=1
    MODEL.addConstr(sum(variables) == args.card, name=str(cnt))
    f=time.time()
    
    
    print('??')
    MODEL.optimize()
    print('??')
    return np.array(MODEL.x)
class myenv2(gym.Env):
    metadata = {
        'render.modes': ['human', 'rgb_array'],
        'video.frames_per_second': 50
    }

    def __init__(self):

        
        

        self.steps = torch.tensor(np.array([0.1]*args.solDim))
        self._max_episode_steps = 500
        self.iteration=1
        self.constraints=[]
        self.constraint=0
        self.history=np.ones(args.solDim)
    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def step(self, action):
        self.iteration+=1 
        done=1
        action=[action[i]+0.0*np.sqrt(np.log(self.iteration)/self.history[i]) for i in range(args.solDim)]
        action=np.array([1 if i in np.argsort(action)[-args.card:] else 0  for i in range(args.solDim)])

        
  
   
    
        
        
        reward=hwithoutconstraints(action)-10*checkFea(action)
        rewards.append(hwithoutconstraints(action))
        constraints.append(checkFea(action))
        self.history+=np.array(action)
        print(self.iteration,1*hwithoutconstraints(action),checkFea(action))
        
        return action, reward, done, {}

    def reset(self):
        return  np.array(np.array([0.1]*args.solDim)),

    def render(self, mode='human'):
        pass


In [15]:
criterion = nn.MSELoss()
class g2apara:
    def __init__(self):
        self.rnn_hidden_dim=8
        self.n_agents=args.solDim
        self.attention_dim=8
        self.cuda=False
        self.n_actions=2
        self.hard=False
args3=g2apara()

In [21]:

hids=torch.from_numpy(np.random.random([args.solDim,args3.rnn_hidden_dim])).float()
rewards=[]
constraints=[]
import gym


class Normalizedenv2(gym.ActionWrapper):
    """ Wrap action """

    def action(self, action):
        act_k = (self.action_space.high - self.action_space.low)/ 2.
        act_b = (self.action_space.high + self.action_space.low)/ 2.
        return act_k * action + act_b

    def reverse_action(self, action):
        act_k_inv = 2./(self.action_space.high - self.action_space.low)
        act_b = (self.action_space.high + self.action_space.low)/ 2.
        return act_k_inv * (action - act_b)
import os
import torch
from torch.autograd import Variable

USE_CUDA = torch.cuda.is_available()
FLOAT = torch.cuda.FloatTensor if USE_CUDA else torch.FloatTensor


def to_numpy(var):
    return var.cpu().data.numpy() if USE_CUDA else var.data.numpy()

def to_tensor(ndarray, volatile=False, requires_grad=False, dtype=FLOAT):
    return Variable(
        torch.from_numpy(ndarray), volatile=volatile, requires_grad=requires_grad
    ).type(dtype)

def soft_update(target, source, tau):
    for target_param, param in zip(target.parameters(), source.parameters()):
        target_param.data.copy_(
            target_param.data * (1.0 - tau) + param.data * tau
        )

def hard_update(target, source):
    for target_param, param in zip(target.parameters(), source.parameters()):
            target_param.data.copy_(param.data)

def get_output_folder(parent_dir, env2_name):
    """Return save folder.
    Assumes folders in the parent_dir have suffix -run{run
    number}. Finds the highest run number and sets the output folder
    to that number + 1. This is just convenient so that if you run the
    same script multiple times tensorboard can plot all of the results
    on the same plots with different names.
    Parameters
    ----------
    parent_dir: str
      Path of the directory containing all experiment runs.
    Returns
    -------
    parent_dir/run_dir
      Path to this run's save directory.
    """
    os.makedirs(parent_dir, exist_ok=True)
    experiment_id = 0
    for folder_name in os.listdir(parent_dir):
        if not os.path.isdir(os.path.join(parent_dir, folder_name)):
            continue
        try:
            folder_name = int(folder_name.split('-run')[-1])
            if folder_name > experiment_id:
                experiment_id = folder_name
        except:
            pass
    experiment_id += 1

    parent_dir = os.path.join(parent_dir, env2_name)
    parent_dir = parent_dir + '-run{}'.format(experiment_id)
    os.makedirs(parent_dir, exist_ok=True)
    return parent_dir
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import savemat
class Evaluator(object):

    def __init__(self, num_episodes, interval, save_path='', max_episode_length=None):
        self.num_episodes = num_episodes
        self.max_episode_length = max_episode_length
        self.interval = interval
        self.save_path = save_path
        self.results = np.array([]).reshape(num_episodes,0)

    def __call__(self, env2, policy, debug=False, visualize=False, save=True):

        self.is_training = False
        observation = None
        result = []

        for episode in range(self.num_episodes):

            
            observation = env2.reset()
            episode_steps = 0
            episode_reward = 0.

            assert observation is not None

            
            done = False
            while not done:
                
                action = policy(observation)

                observation, reward, done, info = env2.step(action)
                if self.max_episode_length and episode_steps >= self.max_episode_length -1:
                    done = True

                
                episode_reward += reward
                episode_steps += 1
            result.append(episode_reward)

        result = np.array(result).reshape(-1,1)
        self.results = np.hstack([self.results, result])

        if save:
            self.save_results('{}/validate_reward'.format(self.save_path))
        return np.mean(result)

    def save_results(self, fn):

        y = np.mean(self.results, axis=0)
        error=np.std(self.results, axis=0)

        x = range(0,self.results.shape[1]*self.interval,self.interval)
        fig, ax = plt.subplots(1, 1, figsize=(6, 5))
        plt.xlabel('Timestep')
        plt.ylabel('Average Reward')
        ax.errorbar(x, y, yerr=error, fmt='-o')
        plt.savefig(fn+'.png')
        savemat(fn+'.mat', {'reward':self.results})
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F



def fanin_init(size, fanin=None):
    fanin = fanin or size[0]
    v = 1. / np.sqrt(fanin)
    return torch.Tensor(size).uniform_(-v, v)

class Actor(nn.Module):
    def __init__(self, input_shape, args):
        super(Actor, self).__init__()

        
        self.encoding = nn.Linear(input_shape, args.rnn_hidden_dim)  
        self.h = nn.GRUCell(args.rnn_hidden_dim, args.rnn_hidden_dim)  

        
        
        
        
        self.hard_bi_GRU = nn.GRU(args.rnn_hidden_dim * 2, args.rnn_hidden_dim, bidirectional=True)
        
        self.hard_encoding = nn.Linear(args.rnn_hidden_dim * 2, 2)  

        
        self.q = nn.Linear(args.rnn_hidden_dim, args.attention_dim, bias=False)
        self.k = nn.Linear(args.rnn_hidden_dim, args.attention_dim, bias=False)
        self.v = nn.Linear(args.rnn_hidden_dim, args.attention_dim)

        
        self.decoding = nn.Linear(args.rnn_hidden_dim + args.attention_dim, args.n_actions)
        self.args = args
        self.input_shape = input_shape
        self.softmax=F.softmax

    def forward(self,state, obs=torch.tensor([1.]*args.solDim).view(args.solDim,1), hidden_state=hids):
        size = obs.shape[0]  
        
        obs_encoding = F.relu(self.encoding(obs))
        h_in = hidden_state.reshape(-1, self.args.rnn_hidden_dim)

        
        h_out = self.h(obs_encoding, h_in)  

        
        if self.args.hard:
            
            h = h_out.reshape(-1, self.args.n_agents, self.args.rnn_hidden_dim)  
            input_hard = []
            for i in range(self.args.n_agents):
                h_i = h[:, i]  
                h_hard_i = []
                for j in range(self.args.n_agents):  
                    if j != i:
                        h_hard_i.append(torch.cat([h_i, h[:, j]], dim=-1))
                
                h_hard_i = torch.stack(h_hard_i, dim=0)
                input_hard.append(h_hard_i)
            
            input_hard = torch.stack(input_hard, dim=-2)
            
            input_hard = input_hard.view(self.args.n_agents - 1, -1, self.args.rnn_hidden_dim * 2)

            h_hard = torch.zeros((2 * 1, size, self.args.rnn_hidden_dim))  
            if self.args.cuda:
                h_hard = h_hard.cuda()
            h_hard, _ = self.hard_bi_GRU(input_hard, h_hard)  
            h_hard = h_hard.permute(1, 0, 2)  
            h_hard = h_hard.reshape(-1, self.args.rnn_hidden_dim * 2)  

            
            hard_weights = self.hard_encoding(h_hard)
            hard_weights = F.gumbel_softmax(hard_weights, tau=0.01)
            
            hard_weights = hard_weights[:, 1].view(-1, self.args.n_agents, 1, self.args.n_agents - 1)
            hard_weights = hard_weights.permute(1, 0, 2, 3)

        else:
            hard_weights = torch.ones((self.args.n_agents, size // self.args.n_agents, 1, self.args.n_agents - 1))
            if self.args.cuda:
                hard_weights = hard_weights.cuda()

        
        q = self.q(h_out).reshape(-1, self.args.n_agents, self.args.attention_dim)  
        k = self.k(h_out).reshape(-1, self.args.n_agents, self.args.attention_dim)  
        v = F.relu(self.v(h_out)).reshape(-1, self.args.n_agents, self.args.attention_dim)  
        x = []
        for i in range(self.args.n_agents):
            q_i = q[:, i].view(-1, 1, self.args.attention_dim)  
            k_i = [k[:, j] for j in range(self.args.n_agents) if j != i]  
            v_i = [v[:, j] for j in range(self.args.n_agents) if j != i]  

            k_i = torch.stack(k_i, dim=0)  
            k_i = k_i.permute(1, 2, 0)  
            v_i = torch.stack(v_i, dim=0)
            v_i = v_i.permute(1, 2, 0)

            
            score = torch.matmul(q_i, k_i)

            
            scaled_score = score / np.sqrt(self.args.attention_dim)

            
            soft_weight = F.softmax(scaled_score, dim=-1)  

            
            x_i = (v_i * soft_weight * hard_weights[i]).sum(dim=-1)
            x.append(x_i)

        
        x = torch.stack(x, dim=1).reshape(-1, self.args.attention_dim)  
        final_input = torch.cat([h_out, x], dim=-1)
        output = self.softmax(1*self.decoding(final_input))
        return torch.cat(([i[0].view(1) for i in output]),0)

class Critic(nn.Module):
    def __init__(self, nb_states, nb_actions, hidden1=400, hidden2=300, init_w=3e-3):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(nb_states, hidden1)
        self.fc2 = nn.Linear(hidden1 +nb_actions, hidden2)
        self.fc3 = nn.Linear(hidden2, 1)
        self.relu = nn.ReLU()
        self.init_weights(init_w)

    def init_weights(self, init_w):
        self.fc1.weight.data = fanin_init(self.fc1.weight.data.size())
        self.fc2.weight.data = fanin_init(self.fc2.weight.data.size())
        self.fc3.weight.data.uniform_(-init_w, init_w)

    def forward(self, xs):
        x, a = xs
        out = self.fc1(x)
        out = self.relu(out)
        
        if len(a.shape)==1:
            a=a.view([1,a.shape[0]])
        out = self.fc2(torch.cat([out,a],1))
        out = self.relu(out)
        out = self.fc3(out)
        return out

from collections import deque, namedtuple
import warnings
import random

import numpy as np





Experience = namedtuple('Experience', 'state0, action, reward, state1, terminal1')


def sample_batch_indexes(low, high, size):
    if high - low >= size:
        
        
        
        
        try:
            r = xrange(low, high)
        except NameError:
            r = range(low, high)
        batch_idxs = random.sample(r, size)
    else:
        
        
        
        warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
        batch_idxs = np.random.random_integers(low, high - 1, size=size)
    assert len(batch_idxs) == size
    return batch_idxs


class RingBuffer(object):
    def __init__(self, maxlen):
        self.maxlen = maxlen
        self.start = 0
        self.length = 0
        self.data = [None for _ in range(maxlen)]

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        if idx < 0 or idx >= self.length:
            raise KeyError()
        return self.data[(self.start + idx) % self.maxlen]

    def append(self, v):
        if self.length < self.maxlen:
            
            self.length += 1
        elif self.length == self.maxlen:
            
            self.start = (self.start + 1) % self.maxlen
        else:
            
            raise RuntimeError()
        self.data[(self.start + self.length - 1) % self.maxlen] = v


def zeroed_observation(observation):
    if hasattr(observation, 'shape'):
        return np.zeros(observation.shape)
    elif hasattr(observation, '__iter__'):
        out = []
        for x in observation:
            out.append(zeroed_observation(x))
        return out
    else:
        return 0.


class Memory(object):
    def __init__(self, window_length, ignore_episode_boundaries=False):
        self.window_length = window_length
        self.ignore_episode_boundaries = ignore_episode_boundaries

        self.recent_observations = deque(maxlen=window_length)
        self.recent_terminals = deque(maxlen=window_length)

    def sample(self, batch_size, batch_idxs=None):
        raise NotImplementedError()

    def append(self, observation, action, reward, terminal, training=True):
        self.recent_observations.append(observation)
        self.recent_terminals.append(terminal)

    def get_recent_state(self, current_observation):
        
        
        
        state = [current_observation]
        idx = len(self.recent_observations) - 1
        for offset in range(0, self.window_length - 1):
            current_idx = idx - offset
            current_terminal = self.recent_terminals[current_idx - 1] if current_idx - 1 >= 0 else False
            if current_idx < 0 or (not self.ignore_episode_boundaries and current_terminal):
                
                
                break
            state.insert(0, self.recent_observations[current_idx])
        while len(state) < self.window_length:
            state.insert(0, zeroed_observation(state[0]))
        return state

    def get_config(self):
        config = {
            'window_length': self.window_length,
            'ignore_episode_boundaries': self.ignore_episode_boundaries,
        }
        return config

class SequentialMemory(Memory):
    def __init__(self, limit, **kwargs):
        super(SequentialMemory, self).__init__(**kwargs)

        self.limit = limit

        
        
        self.actions = RingBuffer(limit)
        self.rewards = RingBuffer(limit)
        self.terminals = RingBuffer(limit)
        self.observations = RingBuffer(limit)

    def sample(self, batch_size, batch_idxs=None):
        if batch_idxs is None:
            
            
            batch_idxs = sample_batch_indexes(0, self.nb_entries - 1, size=batch_size)
        batch_idxs = np.array(batch_idxs) + 1
        assert np.min(batch_idxs) >= 1
        assert np.max(batch_idxs) < self.nb_entries
        assert len(batch_idxs) == batch_size

        
        experiences = []
        for idx in batch_idxs:
            terminal0 = self.terminals[idx - 2] if idx >= 2 else False
            while terminal0:
                
                
                
                idx = sample_batch_indexes(1, self.nb_entries, size=1)[0]
                terminal0 = self.terminals[idx - 2] if idx >= 2 else False
            assert 1 <= idx < self.nb_entries

            
            
            
            state0 = [self.observations[idx - 1]]
            for offset in range(0, self.window_length - 1):
                current_idx = idx - 2 - offset
                current_terminal = self.terminals[current_idx - 1] if current_idx - 1 > 0 else False
                if current_idx < 0 or (not self.ignore_episode_boundaries and current_terminal):
                    
                    
                    break
                state0.insert(0, self.observations[current_idx])
            while len(state0) < self.window_length:
                state0.insert(0, zeroed_observation(state0[0]))
            action = self.actions[idx - 1]
            reward = self.rewards[idx - 1]
            terminal1 = self.terminals[idx - 1]

            
            
            
            state1 = [np.copy(x) for x in state0[1:]]
            state1.append(self.observations[idx])

            assert len(state0) == self.window_length
            assert len(state1) == len(state0)
            experiences.append(Experience(state0=state0, action=action, reward=reward,
                                          state1=state1, terminal1=terminal1))
        assert len(experiences) == batch_size
        return experiences

    def sample_and_split(self, batch_size, batch_idxs=None):
        experiences = self.sample(batch_size, batch_idxs)

        state0_batch = []
        reward_batch = []
        action_batch = []
        terminal1_batch = []
        state1_batch = []
        for e in experiences:
            state0_batch.append(e.state0)
            state1_batch.append(e.state1)
            reward_batch.append(e.reward)
            action_batch.append(e.action)
            terminal1_batch.append(0. if e.terminal1 else 1.)

        
        state0_batch = np.array(state0_batch).reshape(batch_size,-1)
        state1_batch = np.array(state1_batch).reshape(batch_size,-1)
        terminal1_batch = np.array(terminal1_batch).reshape(batch_size,-1)
        reward_batch = np.array(reward_batch).reshape(batch_size,-1)
        action_batch = np.array(action_batch).reshape(batch_size,-1)

        return state0_batch, action_batch, reward_batch, state1_batch, terminal1_batch


    def append(self, observation, action, reward, terminal, training=True):
        super(SequentialMemory, self).append(observation, action, reward, terminal, training=training)

        
        
        if training:
            self.observations.append(observation)
            self.actions.append(action)
            self.rewards.append(reward)
            self.terminals.append(terminal)

    @property
    def nb_entries(self):
        return len(self.observations)

    def get_config(self):
        config = super(SequentialMemory, self).get_config()
        config['limit'] = self.limit
        return config


class EpisodeParameterMemory(Memory):
    def __init__(self, limit, **kwargs):
        super(EpisodeParameterMemory, self).__init__(**kwargs)
        self.limit = limit

        self.params = RingBuffer(limit)
        self.intermediate_rewards = []
        self.total_rewards = RingBuffer(limit)

    def sample(self, batch_size, batch_idxs=None):
        if batch_idxs is None:
            batch_idxs = sample_batch_indexes(0, self.nb_entries, size=batch_size)
        assert len(batch_idxs) == batch_size

        batch_params = []
        batch_total_rewards = []
        for idx in batch_idxs:
            batch_params.append(self.params[idx])
            batch_total_rewards.append(self.total_rewards[idx])
        return batch_params, batch_total_rewards

    def append(self, observation, action, reward, terminal, training=True):
        super(EpisodeParameterMemory, self).append(observation, action, reward, terminal, training=training)
        if training:
            self.intermediate_rewards.append(reward)

    def finalize_episode(self, params):
        total_reward = sum(self.intermediate_rewards)
        self.total_rewards.append(total_reward)
        self.params.append(params)
        self.intermediate_rewards = []

    @property
    def nb_entries(self):
        return len(self.total_rewards)

    def get_config(self):
        config = super(SequentialMemory, self).get_config()
        config['limit'] = self.limit
        return config
import numpy as np 



class RandomProcess(object):
    def reset_states(self):
        pass

class AnnealedGaussianProcess(RandomProcess):
    def __init__(self, mu, sigma, sigma_min, n_steps_annealing):
        self.mu = mu
        self.sigma = sigma
        self.n_steps = 0

        if sigma_min is not None:
            self.m = -float(sigma - sigma_min) / float(n_steps_annealing)
            self.c = sigma
            self.sigma_min = sigma_min
        else:
            self.m = 0.
            self.c = sigma
            self.sigma_min = sigma

    @property
    def current_sigma(self):
        sigma = max(self.sigma_min, self.m * float(self.n_steps) + self.c)
        return sigma



class OrnsteinUhlenbeckProcess(AnnealedGaussianProcess):
    def __init__(self, theta, mu=0., sigma=1., dt=1e-2, x0=None, size=1, sigma_min=None, n_steps_annealing=1000):
        super(OrnsteinUhlenbeckProcess, self).__init__(mu=mu, sigma=sigma, sigma_min=sigma_min, n_steps_annealing=n_steps_annealing)
        self.theta = theta
        self.mu = mu
        self.dt = dt
        self.x0 = x0
        self.size = size
        self.reset_states()

    def sample(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + self.current_sigma * np.sqrt(self.dt) * np.random.normal(size=self.size)
        self.x_prev = x
        self.n_steps += 1
        return x

    def reset_states(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros(self.size)


criterion = nn.MSELoss()

class DDPG(object):
    def __init__(self, nb_states, nb_actions, args2):

        if args2.seed > 0:
            self.seed(args2.seed)

        self.nb_states = nb_states
        self.nb_actions= nb_actions

        
        net_cfg = {
            'hidden1':args2.hidden1, 
            'hidden2':args2.hidden2, 
            'init_w':args2.init_w
        }
        self.actor = Actor(1,args3)
        self.actor_target = Actor(1,args3)
        self.actor_optim  = Adam(self.actor.parameters(), lr=args2.prate)

        self.critic = Critic(self.nb_states, self.nb_actions, **net_cfg)
        self.critic_target = Critic(self.nb_states, self.nb_actions, **net_cfg)
        self.critic_optim  = Adam(self.critic.parameters(), lr=args2.rate)

        hard_update(self.actor_target, self.actor) 
        hard_update(self.critic_target, self.critic)

        
        self.memory = SequentialMemory(limit=args2.rmsize, window_length=args2.window_length)
        self.random_process = OrnsteinUhlenbeckProcess(size=nb_actions, theta=args2.ou_theta, mu=args2.ou_mu, sigma=args2.ou_sigma)

        
        self.batch_size = args2.bsize
        self.tau = args2.tau
        self.discount = args2.discount
        self.depsilon = 1.0 / args2.epsilon

        
        self.epsilon = 1.0
        self.s_t = None 
        self.a_t = None 
        self.is_training = True

        
        if USE_CUDA: self.cuda()

    def update_policy(self):
        
        state_batch, action_batch, reward_batch, \
        next_state_batch, terminal_batch = self.memory.sample_and_split(self.batch_size)

        
        next_q_values = self.critic_target([
            to_tensor(next_state_batch, volatile=True),
            torch.cat([self.actor_target(to_tensor(next_state_batch[i], volatile=True)) for i in range(len(next_state_batch))],0).view([len(next_state_batch),-1]),
        ])
        next_q_values.volatile=False

        target_q_batch = to_tensor(reward_batch) + \
            self.discount*to_tensor(terminal_batch.astype(np.float))*next_q_values

        
        self.critic.zero_grad()

        q_batch = self.critic([ to_tensor(state_batch), to_tensor(action_batch) ])

        value_loss = criterion(q_batch, target_q_batch)
        value_loss.backward()
        self.critic_optim.step()

        
        self.actor.zero_grad()

        policy_loss = -self.critic([
            to_tensor(state_batch),
            torch.cat([self.actor(to_tensor(next_state_batch[i], volatile=True)) for i in range(len(next_state_batch))],0).view([len(state_batch),-1]), ])
        

        policy_loss = policy_loss.mean()
        policy_loss.backward()
        self.actor_optim.step()

        
        soft_update(self.actor_target, self.actor, self.tau)
        soft_update(self.critic_target, self.critic, self.tau)

    def eval(self):
        self.actor.eval()
        self.actor_target.eval()
        self.critic.eval()
        self.critic_target.eval()

    def cuda(self):
        self.actor.cuda()
        self.actor_target.cuda()
        self.critic.cuda()
        self.critic_target.cuda()

    def observe(self, r_t, s_t1, done):
        if self.is_training:
            self.memory.append(self.s_t, self.a_t, r_t, done)
            self.s_t = s_t1

    def random_action(self):
        action = np.random.uniform(0.,1.,self.nb_actions)
        self.a_t = action
        return action

    def select_action(self, s_t, decay_epsilon=True):
        action = to_numpy(
            self.actor(to_tensor(np.array([s_t])))
        )
        
        action = np.clip(action, 0., 1.)

        if decay_epsilon:
            self.epsilon -= self.depsilon

        self.a_t = action

        return action

    def reset(self, obs):
        self.s_t = obs
        self.random_process.reset_states()

    def load_weights(self, output):
        if output is None: return

        self.actor.load_state_dict(
            torch.load('{}/actor.pkl'.format(output))
        )

        self.critic.load_state_dict(
            torch.load('{}/critic.pkl'.format(output))
        )


    def save_model(self,output):
        torch.save(
            self.actor.state_dict(),
            '{}/actor.pkl'.format(output)
        )
        torch.save(
            self.critic.state_dict(),
            '{}/critic.pkl'.format(output)
        )

    def seed(self,s):
        torch.manual_seed(s)
        if USE_CUDA:
            torch.cuda.manual_seed(s)
import pyflann
from gym.spaces import Box
import numpy as np
import itertools

class Space:

    def __init__(self, low, high, points):

        self._low = np.array(low)
        self._high = np.array(high)
        self._range = self._high - self._low
        self._dimensions = len(low)
        self.__space = init_uniform_space([0] * self._dimensions,
                                          [1] * self._dimensions,
                                          points)
        self._flann = pyflann.FLANN()
        self.rebuild_flann()

    def rebuild_flann(self):
        self._index = self._flann.build_index(self.__space, algorithm='kdtree')
        

    def search_point(self, point, k):
        p_in = self.import_point(point).reshape(1, -1).astype('float64')
        
        search_res, _ = self._flann.nn_index(p_in, k)
        knns = self.__space[search_res]
        p_out = []
        for p in knns:
            p_out.append(self.export_point(p))

        if k == 1:
            p_out = [p_out]
        
        return np.array(p_out)

    def import_point(self, point):
        return (point - self._low) / self._range

    def export_point(self, point):
        return self._low + point * self._range

    def get_space(self):
        return self.__space

    def shape(self):
        return self.__space.shape

    def get_number_of_actions(self):
        return self.shape()[0]

    def plot_space(self, additional_points=None):

        dims = self._dimensions

        if dims > 3:
            print(
                'Cannot plot a {}-dimensional space. Max 3 dimensions'.format(dims))
            return

        space = self.get_space()
        if additional_points is not None:
            for i in additional_points:
                space = np.append(space, additional_points, axis=0)

        if dims == 1:
            for x in space:
                plt.plot([x], [0], 'o')

            plt.show()
        elif dims == 2:
            for x, y in space:
                plt.plot([x], [y], 'o')

            plt.show()
        else:
            plot_3d_points(space)


class Discrete_space(Space):
    """
        Discrete action space with n actions (the integers in the range [0, n))
        0, 1, 2, ..., n-2, n-1
    """

    def __init__(self, n):  
        super().__init__([0], [n - 1], n)

    def export_point(self, point):
        return super().export_point(point).astype(int)


def init_uniform_space(low, high, points):
    dims = len(low)
    points_in_each_axis = round(points**(1 / dims))

    axis = []
    for i in range(dims):
        axis.append(list(np.linspace(low[i], high[i], points_in_each_axis)))
    print('points_in_each_axis')
    space = []
    for _ in itertools.product(*axis):
        print('points_in_each_axis')
        space.append(list(_))
    print('points_in_each_axis')
    return np.array(space)
import numpy as np

import torch
import torch.nn as nn
from torch.optim import Adam
criterion = nn.MSELoss()

class WOLPAgent(object):
    def __init__(self, nb_states, nb_actions, args2):

        if args2.seed > 0:
            self.seed(args2.seed)

        self.nb_states = nb_states
        self.nb_actions= nb_actions

        
        net_cfg = {
            'hidden1':args2.hidden1, 
            'hidden2':args2.hidden2, 
            'init_w':args2.init_w
        }

        
        self.low = args2.low
        self.high = args2.high
        self.action_space = spaces.Box(
            low=0,
            high=1,
            shape=(args.solDim,)
        )
        self.k_nearest_neighbors = max(1, int(args2.max_actions * args2.k_ratio))
        

        self.actor =Actor(1,args3)
        self.actor_target = Actor(1,args3)
        self.actor_optim  = Adam(self.actor.parameters(), lr=args2.prate)
        self.critic = Critic(self.nb_states, self.nb_actions, **net_cfg)
        self.critic_target = Critic(self.nb_states, self.nb_actions, **net_cfg)
        self.critic_optim  = Adam(self.critic.parameters(), lr=args2.rate)

        hard_update(self.actor_target, self.actor) 
        hard_update(self.critic_target, self.critic)

        
        self.memory = SequentialMemory(limit=args2.rmsize, window_length=args2.window_length)
        self.random_process = OrnsteinUhlenbeckProcess(size=nb_actions, theta=args2.ou_theta, mu=args2.ou_mu, sigma=args2.ou_sigma)

        
        self.batch_size = args2.bsize
        self.tau = args2.tau
        self.discount = args2.discount
        self.depsilon = 1.0 / args2.epsilon

        
        self.epsilon = 1.0
        self.s_t = None 
        self.a_t = None 
        self.is_training = True

        
        if USE_CUDA: self.cuda()

    def get_action_space(self):
        return self.action_space    

    def update_policy(self):
        
        state_batch, action_batch, reward_batch, \
        next_state_batch, terminal_batch = self.memory.sample_and_split(self.batch_size)

        
        next_q_values = self.critic_target([
            to_tensor(next_state_batch, volatile=True),
            torch.cat([self.actor_target(to_tensor(next_state_batch[i], volatile=True)) for i in range(len(next_state_batch))],0).view([len(next_state_batch),-1]),
        ])
        next_q_values.volatile=False

        target_q_batch = to_tensor(reward_batch) + \
            self.discount*to_tensor(terminal_batch.astype(np.float))*next_q_values

        
        self.critic.zero_grad()
        q_batch = self.critic([ to_tensor(state_batch), to_tensor(action_batch) ])

        value_loss = criterion(q_batch, target_q_batch)
        value_loss.backward()
        self.critic_optim.step()

        
        self.actor.zero_grad()

        policy_loss = -self.critic([
            to_tensor(state_batch),
            torch.cat([self.actor(to_tensor(next_state_batch[i], volatile=True)) for i in range(len(next_state_batch))],0).view([len(state_batch),-1]), ])

        policy_loss = policy_loss.mean()
        policy_loss.backward()
        self.actor_optim.step()

        
        soft_update(self.actor_target, self.actor, self.tau)
        soft_update(self.critic_target, self.critic, self.tau)

    def eval(self):
        self.actor.eval()
        self.actor_target.eval()
        self.critic.eval()
        self.critic_target.eval()

    def cuda(self):
        self.actor.cuda()
        self.actor_target.cuda()
        self.critic.cuda()
        self.critic_target.cuda()

    def observe(self, r_t, s_t1, done):
        if self.is_training:
            self.memory.append(self.s_t, self.a_t, r_t, done)
            self.s_t = s_t1

    def random_action(self):
        action = np.random.uniform(0.,1.,self.nb_actions)
        self.a_t = action
        return action

    def select_action(self, s_t, decay_epsilon=True):

        return self.ddpg_select_action(s_t, decay_epsilon=decay_epsilon)

    def ddpg_select_action(self, s_t, decay_epsilon=True):
        action = to_numpy(
            self.actor(to_tensor(np.array([s_t])))
        )
        
        action = np.clip(action, 0., 1.)
        if decay_epsilon:
            self.epsilon -= self.depsilon

        return action

    def reset(self, obs):
        self.s_t = obs
        self.random_process.reset_states()

    def load_weights(self, output):
        if output is None: return

        self.actor.load_state_dict(
            torch.load('{}/actor.pkl'.format(output))
        )

        self.critic.load_state_dict(
            torch.load('{}/critic.pkl'.format(output))
        )


    def save_model(self,output):
        torch.save(
            self.actor.state_dict(),
            '{}/actor.pkl'.format(output)
        )
        torch.save(
            self.critic.state_dict(),
            '{}/critic.pkl'.format(output)
        )

    def seed(self,s):
        torch.manual_seed(s)
        if USE_CUDA:
            torch.cuda.manual_seed(s)
import math
import gym
from gym import spaces, logger
from gym.utils import seeding
import numpy as np

def test(num_episodes, agent, env2, evaluate, model_path, visualize=True, debug=False):

    agent.load_weights(model_path)
    agent.is_training = False
    agent.eval()
    policy = lambda x: agent.select_action(x, decay_epsilon=False)

    for i in range(num_episodes):
        validate_reward = evaluate(env2, policy, debug=debug, visualize=visualize, save=False)
class Arguments(object):
    def __init__(self):
        self.mode = 'train'
        self.env2 = "InvertedPendulum-v2"
        self.hidden1 = 400
        self.hidden2 = 300
        self.rate = 0.001
        self.prate = 0.0001
        self.warmup = 100
        self.discount = 0.99
        self.bsize = 64
        self.rmsize = 6000000
        self.window_length = 1
        self.tau = 0.001
        self.ou_theta = 0.15
        self.ou_sigma = 0.2
        self.ou_mu = 0.0
        self.validate_episodes = 20
        self.max_episode_length = 500
        self.validate_steps = 50000
        self.output = 'output'
        self.debug='debug'
        self.init_w = 0.03
        self.train_iter=5000
        self.epsilon=50000
        self.seed=-1
        self.max_actions=1e6
        self.resume='default'
        self.k_ratio = 1e-6

args2 = Arguments()
args2.output = get_output_folder(args2.output, args2.env2)
if args2.resume == 'default':
    args2.resume = 'output/{}-run0'.format(args2.env2)
class myenv2(gym.Env):
    metadata = {
        'render.modes': ['human', 'rgb_array'],
        'video.frames_per_second': 50
    }

    def __init__(self):

        
        

        self.action_space =  spaces.Box(
            low=0,
            high=1,
            shape=(args.solDim,)
        )
        self.observation_space =  spaces.Box(
            low=0,
            high=1,
            shape=(args.solDim,)
        )

        self.seed()
        self.viewer = None
        self.state = None

        self.steps_beyond_done = None

        self.steps = torch.tensor(np.array([0.1]*args.solDim))
        self._max_episode_steps = 500
        self.iteration=1
        self.constraints=[]
        self.constraint=0
        self.history=np.ones(args.solDim)
    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def step(self, action):
        self.iteration+=1 
        if self.iteration==60:
            self.constraint=10
            self.std=np.array(rewards).std()
        if self.iteration>1 and self.iteration%10==0:
            self.constraint=min(100*sum(rewards)/len(rewards),max(0,self.constraint + 0.005*(-0.02+sum(self.constraints)/len(self.constraints))))
        done=1
        action=[action[i]+0.01*np.sqrt(np.log(self.iteration)/self.history[i]) for i in range(args.solDim)]
        action=np.array([1 if i in np.argsort(action)[-args.card:] else 0  for i in range(args.solDim)])

        self.constraints.append(checkFea(action))
        
  
   
    
        
        
        reward=hwithoutconstraints(action)-self.constraint*checkFea(action)
        rewards.append(hwithoutconstraints(action))
        constraints.append(checkFea(action))
        self.history+=np.array(action)
        print(self.iteration,1*hwithoutconstraints(action),checkFea(action))
        
        return action, reward, done, {}

    def reset(self):
        return  np.array(np.array([0.1]*args.solDim)),

    def render(self, mode='human'):
        pass




env2 = myenv2()

args2.low = env2.action_space.low
args2.high = env2.action_space.high


if args2.seed > 0:
    np.random.seed(args2.seed)
    env2.seed(args2.seed)

nb_states =args.solDim
nb_actions =args.solDim
agent = WOLPAgent(nb_states, nb_actions, args2)

evaluate = Evaluator(args2.validate_episodes, 
    args2.validate_steps, args2.output, max_episode_length=args2.max_episode_length)
num_iterations=args2.train_iter
validate_steps=args2.validate_steps
output= args2.output
max_episode_length=args2.max_episode_length
debug=args2.debug
agent.is_training = True
step = episode = episode_steps = 0
episode_reward = 0.
observation = None

In [22]:
class Actor(nn.Module):
    def __init__(self, input_shape, args):
        super(Actor, self).__init__()

        
        self.encoding = nn.Linear(input_shape, args.rnn_hidden_dim)  
        self.h = nn.GRUCell(args.rnn_hidden_dim, args.rnn_hidden_dim)  

        
        
        
        
        self.hard_bi_GRU = nn.GRU(args.rnn_hidden_dim * 2, args.rnn_hidden_dim, bidirectional=True)
        
        self.hard_encoding = nn.Linear(args.rnn_hidden_dim * 2, 2)  

        
        self.q = nn.Linear(args.rnn_hidden_dim, args.attention_dim, bias=False)
        self.k = nn.Linear(args.rnn_hidden_dim, args.attention_dim, bias=False)
        self.v = nn.Linear(args.rnn_hidden_dim, args.attention_dim)

        
        self.decoding = nn.Linear(args.rnn_hidden_dim + args.attention_dim, args.n_actions)
        self.args = args
        self.input_shape = input_shape
        self.softmax=F.softmax

    def forward(self,state, obs=torch.diag(torch.ones(args.solDim)), hidden_state=hids):
        size = obs.shape[0]  
        
        obs_encoding = F.relu(self.encoding(obs))
        h_in = hidden_state.reshape(-1, self.args.rnn_hidden_dim)

        
        h_out = self.h(obs_encoding, h_in)  

        
        if self.args.hard:
            
            h = h_out.reshape(-1, self.args.n_agents, self.args.rnn_hidden_dim)  
            input_hard = []
            for i in range(self.args.n_agents):
                h_i = h[:, i]  
                h_hard_i = []
                for j in range(self.args.n_agents):  
                    if j != i:
                        h_hard_i.append(torch.cat([h_i, h[:, j]], dim=-1))
                
                h_hard_i = torch.stack(h_hard_i, dim=0)
                input_hard.append(h_hard_i)
            
            input_hard = torch.stack(input_hard, dim=-2)
            
            input_hard = input_hard.view(self.args.n_agents - 1, -1, self.args.rnn_hidden_dim * 2)

            h_hard = torch.zeros((2 * 1, size, self.args.rnn_hidden_dim))  
            if self.args.cuda:
                h_hard = h_hard.cuda()
            h_hard, _ = self.hard_bi_GRU(input_hard, h_hard)  
            h_hard = h_hard.permute(1, 0, 2)  
            h_hard = h_hard.reshape(-1, self.args.rnn_hidden_dim * 2)  

            
            hard_weights = self.hard_encoding(h_hard)
            hard_weights = F.gumbel_softmax(hard_weights, tau=0.01)
            
            hard_weights = hard_weights[:, 1].view(-1, self.args.n_agents, 1, self.args.n_agents - 1)
            hard_weights = hard_weights.permute(1, 0, 2, 3)

        else:
            hard_weights = torch.ones((self.args.n_agents, size // self.args.n_agents, 1, self.args.n_agents - 1))
            if self.args.cuda:
                hard_weights = hard_weights.cuda()

        
        q = self.q(h_out).reshape(-1, self.args.n_agents, self.args.attention_dim)  
        k = self.k(h_out).reshape(-1, self.args.n_agents, self.args.attention_dim)  
        v = F.relu(self.v(h_out)).reshape(-1, self.args.n_agents, self.args.attention_dim)  
        x = []
        for i in range(self.args.n_agents):
            q_i = q[:, i].view(-1, 1, self.args.attention_dim)  
            k_i = [k[:, j] for j in range(self.args.n_agents) if j != i]  
            v_i = [v[:, j] for j in range(self.args.n_agents) if j != i]  

            k_i = torch.stack(k_i, dim=0)  
            k_i = k_i.permute(1, 2, 0)  
            v_i = torch.stack(v_i, dim=0)
            v_i = v_i.permute(1, 2, 0)

            
            score = torch.matmul(q_i, k_i)

            
            scaled_score = score / np.sqrt(self.args.attention_dim)

            
            soft_weight = F.softmax(scaled_score, dim=-1)  

            
            x_i = (v_i * soft_weight * hard_weights[i]).sum(dim=-1)
            x.append(x_i)

        
        x = torch.stack(x, dim=1).reshape(-1, self.args.attention_dim)  
        final_input = torch.cat([h_out, x], dim=-1)
        output = self.softmax(1*self.decoding(final_input))
        return torch.cat(([i[0].view(1) for i in output]),0)

class Critic(nn.Module):
    def __init__(self, nb_actions, hidden1=400, hidden2=300, init_w=3e-3):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(nb_actions, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, 1)
        self.relu = nn.ReLU()
        self.init_weights(init_w)

    def init_weights(self, init_w):
        self.fc1.weight.data = fanin_init(self.fc1.weight.data.size())
        self.fc2.weight.data = fanin_init(self.fc2.weight.data.size())
        self.fc3.weight.data.uniform_(-init_w, init_w)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out


In [ ]:

actor=Actor(args.solDim,args3)
critic=Critic(args.solDim)
criterion = nn.MSELoss()
opt1 =torch.optim.Adam(actor.parameters(), lr=1e-3)
opt2 =torch.optim.Adam(critic.parameters(), lr=1e-3)
iteration=0
history=np.ones(args.solDim)
constraints=[]
constraint=0
xs=[]
ys=[]
for iteration in range(5001):
    if iteration<100:
        action=[1.]*args.card+[0]*(args.solDim-args.card)
        np.random.shuffle(action)
        xs=[action]
        action=np.array(action)
        ys=[hwithoutconstraints(action)-constraint*checkFea(action)]
        rewards.append(hwithoutconstraints(action))
        constraints.append(checkFea(action))
    else:
        action=actor(1)

        if iteration==60:
            constraint=10#100*sum(rewards)/len(rewards)
            std=np.array(rewards).std()
        if iteration>1 and iteration%60==0:
            constraint=min(100*sum(rewards)/len(rewards),max(0,constraint + 0.005*(-0.0+sum(constraints)/len(constraints))))
        done=1
        action2=[action[i]+0.0002*np.sqrt(np.log(iteration)/history[i]) for i in range(args.solDim)]
        action2=np.array([1 if i in np.argsort(action2)[-args.card:] else 0  for i in range(args.solDim)])
        print(iteration,hwithoutconstraints(action2)-10*checkFea(action2))
        rewards.append(hwithoutconstraints(action2))
        constraints.append(checkFea(action2))
        history+=np.array(action2)
        loss=-critic(action)
        opt1.zero_grad()
        loss.backward()
        opt1.step()
        xs=[action]
        ys=[hwithoutconstraints(action2)-constraint*checkFea(action2)]
    if iteration>0:
        loss2=criterion(critic(torch.tensor(xs[-1])),torch.tensor(ys[-1]).float())
        opt2.zero_grad()
        loss2.backward()
        opt2.step()
    if iteration%1000==0:
        np.save('reward_g2a_realorderfm.npy',rewards)
        np.save('constraint_g2a_realorderfm.npy',constraints)

In [25]:
np.save('reward_g2a_realorderfm.npy',rewards+[rewards[-1]]*(5000-len(rewards)))

np.save('constraint_g2a_realorderfm.npy',constraints+[constraints[-1]]*(5000-len(constraints)))

In [ ]:
hwithoutconstraints = lambda x:10*((x.dot(a))**2)  #100*(x.dot(a))**1+100*x.dot(Q).dot(x) # 、100*x.dot(Q).dot(x)

actor=Actor(args.solDim,args3)
critic=Critic(args.solDim)
criterion = nn.MSELoss()
opt1 =torch.optim.Adam(actor.parameters(), lr=1e-3)
opt2 =torch.optim.Adam(critic.parameters(), lr=1e-3)
iteration=0
history=np.ones(args.solDim)
constraints=[]
constraint=0
xs=[]
ys=[]
for iteration in range(5001):
    if iteration<100:
        action=[1.]*args.card+[0]*(args.solDim-args.card)
        np.random.shuffle(action)
        xs=[action]
        action=np.array(action)
        ys=[hwithoutconstraints(action)-constraint*checkFea(action)]
        rewards.append(hwithoutconstraints(action))
        constraints.append(checkFea(action))
    else:
        action=actor(1)

        if iteration==60:
            constraint=10#100*sum(rewards)/len(rewards)
            std=np.array(rewards).std()
        if iteration>1 and iteration%60==0:
            constraint=min(100*sum(rewards)/len(rewards),max(0,constraint + 0.005*(-0.0+sum(constraints)/len(constraints))))
        done=1
        action2=[action[i]+0.0002*np.sqrt(np.log(iteration)/history[i]) for i in range(args.solDim)]
        action2=np.array([1 if i in np.argsort(action2)[-args.card:] else 0  for i in range(args.solDim)])
        print(iteration,hwithoutconstraints(action2)-10*checkFea(action2))
        rewards.append(hwithoutconstraints(action2))
        constraints.append(checkFea(action2))
        history+=np.array(action2)
        loss=-critic(action)
        opt1.zero_grad()
        loss.backward()
        opt1.step()
        xs=[action]
        ys=[hwithoutconstraints(action2)-constraint*checkFea(action2)]
    if iteration>0:
        loss2=criterion(critic(torch.tensor(xs[-1])),torch.tensor(ys[-1]).float())
        opt2.zero_grad()
        loss2.backward()
        opt2.step()
    

In [ ]:
aa=np.random.random(10)
aa

In [ ]:
import numpy as np
def top_k(x, k):
    return np.argpartition(x, k)[..., -k:]

def sample_k(logits, k):
    u = np.random.uniform(size=np.shape(logits))
    z = -np.log(-np.log(u))
    return top_k(logits +z, k)
sample_k(aa,2)